In [1]:
#importing libraries
import folium
import pandas as pd
import numpy as np
import time
import geocoder


In [2]:
# San Francisco latitude and longitude values
lat = 37.77
long = -122.42

In [32]:
sanfran_map=folium.Map(location=[lat, long], zoom_start=12)
sanfran_map

In [3]:
# read in the data
df_streets=pd.read_csv("Potentail_Bust_Stops.csv")

#show the data
df_streets.head()


,Street_One,Street_Two
0,MISSION ST,ITALY AVE
1,MISSION ST,NEW MONTGOMERY ST
2,MISSION ST,01ST ST
3,MISSION ST,20TH ST
4,MISSION ST,FREMONT ST


In [5]:
#shape of the dataset
df_streets.shape

(119, 2)

In [4]:
# rename columns
df_streets.rename(columns={"Street_One": "Str1", "Street_Two":"Str2"}, inplace=True)
df_streets.head()

,Str1,Str2
0,MISSION ST,ITALY AVE
1,MISSION ST,NEW MONTGOMERY ST
2,MISSION ST,01ST ST
3,MISSION ST,20TH ST
4,MISSION ST,FREMONT ST


In [5]:
# Coordinates of the intersection with index 0
inter=df_streets.iloc[0]
intersection=inter[0]+' & '+inter[1] + ", San Francisco, US"
result = geocoder.arcgis(intersection)

In [73]:
print("Address: ", result.address)
print("Latitude: ", result.lat)
print("Longitude: ", result.lng)
print(result.latlng)

Address:  Mission St & Italy Ave, San Francisco, California, 94112
Latitude:  37.71846
Longitude:  -122.4395325
[37.71846, -122.4395325]


In [85]:
# Getting the coordinates of all the places
# This will take a lot of time because of the sleep() function. 
# I had to use this to match the rate limit of arcis
lats, longs=[], []
for str1, str2 in zip(df_streets.Str1, df_streets.Str2):
    intersection=str1 + ' & ' + str2 + ", San Francisco, US"
    time.sleep(0)
    result=geocoder.arcgis(intersection)
    lats.append(result.lat)
    longs.append(result.lng)
    print(result.lat, result.lng, end='\n\n')

37.71846 -122.4395325

37.787445000000005 -122.400504

37.79000927940522 -122.39726220078478

37.758627000000004 -122.419044

37.790469 -122.3966925

37.76997600000001 -122.41992150000002

37.76906700000001 -122.42004299999999

37.791152999999994 -122.395842

37.745577000000004 -122.4198675

37.733949 -122.4261225

37.71765 -122.4401355

37.78837200000001 -122.399334

37.784043 -122.404833

37.72873800000001 -122.431284

37.714608000000005 -122.4426555

37.742382000000006 -122.42196449999999

37.791846 -122.3949735

37.730772 -122.42925450000001

37.76665500000001 -122.41982250000001

37.741743 -122.422383

37.728234 -122.4317835

37.727712000000004 -122.43234600000001

37.73664000000001 -122.42423249999999

37.709541 -122.450517

37.793240999999995 -122.393196

37.70830608338589 -122.45424798660152

37.762713000000005 -122.41946250000001

37.79000927940522 -122.39726220078478

37.782189 -122.407173

37.716381 -122.44100399999999

37.741005 -122.422842

37.755449999999996 -122.41874250

In [88]:
print(len(lats), len(longs))

119 119


In [97]:
# add the latitudes and longitudes
streets=df_streets.copy()
streets['lat']=lats
streets['long']=longs
streets.head()

,Str1,Str2,lat,long
0,MISSION ST,ITALY AVE,37.718460,-122.439532
1,MISSION ST,NEW MONTGOMERY ST,37.787445,-122.400504
2,MISSION ST,01ST ST,37.790009,-122.397262
3,MISSION ST,20TH ST,37.758627,-122.419044
4,MISSION ST,FREMONT ST,37.790469,-122.396693


In [102]:
# initialize a Feature Group for the intersections
intersections=folium.map.FeatureGroup()

# loop through all the intersections and add them to the Feature Group
for lat, lng in zip(streets.lat, streets.long):
    intersections.add_child(
    folium.features.CircleMarker(
    [lat, lng],
    radius=5, # define how big you want the circle markers to be
    color='yellow',
    fill=True,
    fill_color='blue',
    fill_opacity=0.6))

# add intersections to the map
sanfran_map.add_child(intersections)